In [0]:
from pyspark.sql.functions import col, when, count, mean, lit
from pyspark.sql.types import NumericType

In [0]:
df = spark.read.table("formula_1.stg_laps_by_drivers_sessions_meetings")
display(df.limit(5))

In [0]:
numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, NumericType)]
display(df.select(numeric_cols).describe())

In [0]:
display(df.printSchema())

In [0]:
for k, v in df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).collect()[0].asDict().items():
    if v > 0:
        print({"COLUMN": k, "NULL VALUES": v})
    if ('duration_sector' in k or '_speed' in k) and v > 0:
        print(f"Rellenando nulos en: {k}")

        avg_df = (
            df.groupBy("full_name", "session_name")
                .agg(mean(k).alias(f"{k}_mean"))
        )

        df = (
            df.join(avg_df, ["full_name", "session_name"], "left")
                .withColumn(
                    k,
                    when(col(k).isNull(), col(f"{k}_mean"))
                    .otherwise(col(k))
                )
                .drop(f"{k}_mean")
        )
    if 'segments_sector_' in k and v > 0:
        print(f"Rellenando nulos en: {k}")
        df = df.withColumn(k, when(col(k).isNull(), [0]).otherwise(col(k)))
    if 'lap_date_start' in k and v > 0:
        print(f"Rellenando nulos en: {k}")
        df = df.withColumn(k, when(col(k).isNull(), lit(None).cast("timestamp")).otherwise(col(k)))
    if 'country_code' in k and v > 0:
        print(f"Rellenando nulos en: {k}")
        df = df.withColumn(k, when(col(k).isNull(), lit("Unknown")).otherwise(col(k)))

        

In [0]:
df = df.withColumn("lap_duration", when(col("lap_duration").isNull(), col("duration_sector_1")+col("duration_sector_2")+col("duration_sector_3")).otherwise(col("lap_duration")))

In [0]:
for k, v in df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).collect()[0].asDict().items():
    if v > 0:
        print({"COLUMN": k, "NULL VALUES": v})

In [0]:
display(df.groupBy("full_name").count().orderBy("count", ascending=False))

Databricks visualization. Run in Databricks to view.

In [0]:
display(df.limit(5))

In [0]:
for k, v in df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).collect()[0].asDict().items():
    if 'duration_sector' in k or '_speed' in k or 'lap_duration' in k:
        display(df.select(k))

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

...

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# display(df.where(col("driver_country_code")=="Unknown"))
display(df.select("segments_sector_1").distinct())

### Revisamos datos transformados

In [0]:
df_clean = spark.read.table("formula_1.int_transform_laps_data")
display(df_clean.limit(5))

In [0]:
numeric_cols = [f.name for f in df_clean.schema.fields if isinstance(f.dataType, NumericType)]
display(df_clean.select(numeric_cols).describe())

In [0]:
for k, v in df_clean.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_clean.columns
]).collect()[0].asDict().items():
    if v > 0:
        print({"COLUMN": k, "NULL VALUES": v})

In [0]:
for col in df_clean.columns:
    if 'duration_sector' in col or '_speed' in col or 'lap_duration' in col:
        print({"COLUMN": col})
        display(df_clean.select(col))

Databricks visualization. Run in Databricks to view.